In [ ]:
import pandas as pd
from tqdm import tqdm

import numpy

import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import StackingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

import random

import math

from sklearn.metrics import roc_curve, auc  

from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, SVMSMOTE, KMeansSMOTE
from sklearn.model_selection import train_test_split

import xgboost as xgb
import catboost as cb
import shap

import pickle

import settings
dir_path = settings.DIR_PATH

In [ ]:
df = pd.read_pickle(dir_path + 'dataset_v2.pkl')

check = df.loc[(df['target'] == 1)
               & (~df['ref_title'].isna())
            & (df['citation_cnt'].isna())]

tempolary_xml = pd.read_pickle(dir_path + 'tempolary_xml.pkl')
tempolary_xml_ref = pd.read_pickle(dir_path + 'tempolary_xml.pkl')

df = df.merge(tempolary_xml[['_id','title','abstract']],how = 'left',on = '_id')

df['title'] = df.apply(lambda x: x['title_x'] if pd.notna(x['title_x']) else x['title_y'], axis=1)

df.drop(columns=['title_x', 'title_y'], inplace=True)

df['abstract'] = df.apply(lambda x: x['abstract_x'] if pd.notna(x['abstract_x']) else x['abstract_y'], axis=1)

df.drop(columns=['abstract_x', 'abstract_y'], inplace=True)

df['title'] = df['title'].str.replace('\n', '', regex=False)
df['ref_title'] = df['ref_title'].str.replace('\n', '', regex=False)
df['abstract'] = df['abstract'].str.replace('\n', '', regex=False)
df['ref_abstract'] = df['ref_abstract'].str.replace('\n', '', regex=False)

df['abstract'] = df['abstract'].str.lower().str.replace(r'[^\w\s]', '', regex=True)
df['ref_abstract'] = df['ref_abstract'].str.lower().str.replace(r'[^\w\s]', '', regex=True)
df['ref_title'] = df['ref_title'].str.lower().str.replace(r'[^\w\s]', '', regex=True)
df['title'] = df['title'].str.lower().str.replace(r'[^\w\s]', '', regex=True)

def set_flags(df, column, flags):
    for flag, substr in flags.items():
        df[flag] = np.where(df[column].str.contains(substr, case=False, na=False), 1, 0)

flags = {
    'IEEE_flg': 'ieee',
    'acm_flg': 'acm',
    'cvpr_flg': 'cvpr',
    'nips_flg': 'nips',
    'Econometrica_flg':'econometrica',
    'KDD_flg':'kdd'
}

set_flags(df, 'venue', flags)
set_flags(df, 'ref_venue', {f'ref_{k}': v for k, v in flags.items()})

df['Section'] = df['Section'].fillna('')
df['Pre Text'] = df['Pre Text'].fillna('')
df['Post Text'] = df['Post Text'].fillna('')


def set_text_flags(df, columns, patterns):
    for col, flags in patterns.items():
        for flag, substr in flags.items():
            df[flag] = np.where(df[col].str.contains(substr, case=False, na=False), 1, 0)

text_patterns = {
    'Pre Text': {
        'important_flg': 'important',
        'motiva_flg': 'motiva',
        'most_flg': ' most ',
        'impact_flg': 'impact',
        'based_flg': 'based'
    },
    'Post Text': {
        'important_flg': 'important',
        'motiva_flg': 'motiva',
        'most_flg': ' most ',
        'impact_flg': 'impact',
        'based_flg': 'based'
    },
    'Section': {
        'conclusion_flg': 'conclusion',
        'intro_flg': 'intro',
        'material_flg': 'material',
        'method_flg': 'method',
        'result_flg': 'result',
        'discussion_flg': 'discussion',
        'abstract_flg': 'abstract'
    }
}

set_text_flags(df, text_patterns.keys(), text_patterns)


valid_index_o = df.loc[df['train_or_valid'] == 'valid'][['_id','ref_title']].copy()
test_index_o = df.loc[df['train_or_valid'] == 'test'][['_id','ref_title']].copy()

venue_impact = pd.read_pickle(dir_path + 'venue_impact_v2.pkl')
venue_impact.columns = ['venue','v_n_citation']

dblp_feature = pd.read_pickle(dir_path + 'dblp_feature_v2.pkl')


In [ ]:
node2vec = pd.read_csv(dir_path + 'node2vec_umap_v3.csv.gz',compression='gzip')[['UMAP1','UMAP2','node_id']]

In [ ]:
node2vec.columns = ['umap1_node2vec','umap2_node2vec','_id']
df = df.merge(node2vec, on = '_id', how = 'left')
node2vec.columns = ['umap1_node2vec_ref','umap2_node2vec_ref','ref_id']
df = df.merge(node2vec, on = 'ref_id', how = 'left')

author_citation = pd.read_pickle(dir_path + 'author_citation_v2.pkl')

author_citation.columns = ['_id','author_citation']
dblp_feature.columns = ['_id','in_citation_count','page_count']
df = df.merge(author_citation,on = '_id',how = 'left')

dblp_feature = dblp_feature.groupby('_id').sum().reset_index()

#df = df.merge(co_author,on = ['_id','ref_id'],how = 'left')
print(len(df))
df = df.merge(dblp_feature,on = ['_id'],how = 'left')


df.loc[df['Citation Number'] == 'Unknown', 'Citation Number'] = np.nan 

df['citation_cnt'] = df.groupby('_id')['citation_cnt'].transform(lambda x: x.fillna(x.median()))
df['Citation Number'] = df.groupby('_id')['Citation Number'].transform(lambda x: x.fillna(x.median()))

df.loc[df['Citation Number'] == 'Unknown','Citation Number'] = 99
df['Citation Number'] = df['Citation Number'].fillna(99)
df['Citation Number'] = df['Citation Number'].astype(int)
df.loc[df['Citation Number'] >= 99,'Citation Number'] = 99
df.loc[df['Citation Number'] < 1,'Citation Number'] = 99

max_cinum = df[['Citation Number','_id']].loc[df['Citation Number'] != 999].groupby('_id').max().reset_index()
max_cinum.columns = ['_id','max_citation_num']

df = df.merge(max_cinum,how = 'left',on = '_id')

df['Citation_Ration'] = df['Citation Number']/df['max_citation_num']

train_index_o = pd.concat([df[(df['target'] == 1)],# & (df.annotation_ref.isna())],
              df[(df['train_or_valid'] == 'train') & (df['target'] == 0)].sample(n = 7000,random_state= 42)])
train_index_o.to_csv('train_index.csv',index = False)

df['diff_year'] = df['year'] - df['ref_year']
df['n_citation'] = df['n_citation'].apply(lambda x: x + 0.001)
df['ref_n_citation'] = df['ref_n_citation'].apply(lambda x: x + 0.001)
df['diff_year'] = df['diff_year'].apply(lambda x: x + 0.001)

df.loc[df['diff_year'] <= 0, 'diff_year'] = 0


In [ ]:
umap_title = pd.read_pickle(dir_path + 'title_umap_v2.pkl')
umap_abstract = pd.read_pickle(dir_path + 'abstract_umap_v2.pkl')

umap_title = umap_title.rename(columns = {'umap_1':'umap_1_title','umap_2':'umap_2_title',
                                          'umap_ref_1':'umap_ref_1_title','umap_ref_2':'umap_ref_2_title',
                                          'cosine_similarity':'cosine_similarity_title',
                                          'mahalanobis_distance':'mahalanobis_distance_title'})

umap_abstract = umap_abstract.rename(columns = {'umap_1':'umap_1_abstract','umap_2':'umap_2_abstract',
                                              'umap_ref_1':'umap_ref_1_abstract','umap_ref_2':'umap_ref_2_abstract',
                                              'cosine_similarity':'cosine_similarity_abstract',
                                              'mahalanobis_distance':'mahalanobis_distance_abstract'
                                               })

In [ ]:
umap_title = umap_title.groupby(['_id','ref_title']).mean().reset_index()
umap_abstract = umap_abstract.groupby(['_id','ref_title']).mean().reset_index()

print(len(df))
df = df.merge(umap_title,on = ['_id','ref_title'],how = 'left')
df = df.merge(umap_abstract,on = ['_id','ref_title'],how = 'left')
print(len(df))

umap_context = pd.read_pickle(dir_path + 'context_umap.pkl')

umap_context = umap_context.rename(columns = {'umap_1':'umap_1_context','umap_2':'umap_2_context',
                                              'umap_ref_1':'umap_ref_1_context','umap_ref_2':'umap_ref_2_context',
                                              'cosine_similarity':'cosine_similarity_context',
                                              'mahalanobis_distance':'mahalanobis_distance_context'
                                             })

In [7]:
umap_context = umap_context.drop(columns = {'Pre Text','Post Text'}).groupby(['_id','ref_title']).mean().reset_index()
df = df.merge(umap_context,on = ['_id','ref_title'],how = 'left')
print(len(df))

304353


In [8]:

df_temp = df[['citation_cnt', 'Citation Number',
           'ref_n_citation', 'n_citation',
           'train_or_valid', 'target',
           'IEEE_flg', 'acm_flg', 'cvpr_flg', 'nips_flg',
           'Econometrica_flg', 'KDD_flg', 'ref_IEEE_flg', 'ref_acm_flg',
           'ref_cvpr_flg', 'ref_nips_flg', 'ref_Econometrica_flg', 'ref_KDD_flg',
           'important_flg', 'motiva_flg', 'most_flg', 'impact_flg', 'based_flg',
           'conclusion_flg', 'intro_flg', 'material_flg', 'method_flg',
           'result_flg', 'discussion_flg', 'abstract_flg', 
            'umap1_node2vec', 'umap2_node2vec',
              'umap1_node2vec_ref', 'umap2_node2vec_ref',
           'author_citation',
           'in_citation_count', 'page_count', 'max_citation_num',
           'Citation_Ration', 'diff_year', 'umap_1_title',
           'umap_2_title', 'umap_ref_1_title', 'umap_ref_2_title',
           'cosine_similarity_title', 'mahalanobis_distance_title',
           'umap_1_abstract', 'umap_2_abstract', 'umap_ref_1_abstract',
           'umap_ref_2_abstract', 'cosine_similarity_abstract',
           'mahalanobis_distance_abstract', 'umap_1_context', 'umap_2_context',
           'umap_ref_1_context', 'umap_ref_2_context',
           'cosine_similarity_context'
             ]].copy()



In [ ]:

NT = 100

for j in range(0,50):

    for i in range(100,200):
        r = random_integer = random.randint(42, 999)

        try:
            df_train = df_temp.loc[train_index_o.index]
            df_train.head()
            df_train = df_train.reset_index(drop = True)

            df_train['ref_n_citation'] = df_train['ref_n_citation'].astype(float)
            df_train['n_citation'] = df_train['n_citation'].astype(float)

            df_train = df_train.loc[~df_train['cosine_similarity_title'].isna()]
            df_train = df_train.dropna()

            count_class_0, count_class_1 = df_train['target'].value_counts()

            df_class_0 = df_train[df_train['target'] == 0]
            df_class_1 = df_train[df_train['target'] == 1]

            df_class_0_under = df_class_0.sample(int(count_class_1*1.3), random_state = r)
            df_train_under = pd.concat([df_class_0_under, df_class_1], axis=0)
            df_train_under = df_train_under.sample(frac=1).reset_index(drop=True)

            df_train = df_train_under.copy()

            X = df_train.drop(['target','train_or_valid'], axis=1)  
            y = df_train['target'] 

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            blsmote = BorderlineSMOTE(random_state=42)
            X_train_blsmote, y_train_blsmote = blsmote.fit_resample(X_train, y_train)

            print("Borderline-SMOTE resampled data shape:", X_train_blsmote.shape, y_train_blsmote.shape)

            break
        except:
            None

    X_train = X_train_blsmote
    y_train = y_train_blsmote

    def objective(trial):
        param = {
            'objective': 'binary',
            'metric': 'auc',
            'verbosity': -1,
            'boosting_type': 'gbdt',
            'lambda_l1': trial.suggest_float('lambda_l1', 0.7, 1.1),  
            'lambda_l2': trial.suggest_float('lambda_l2', 0.09, 0.17),
            'num_leaves': trial.suggest_int('num_leaves', 150, 350),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.35, 0.55),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.55, 0.7),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 2),
            'min_child_samples': trial.suggest_int('min_child_samples', 80, 110)
        }


        train_data = lgb.Dataset(X_train, label=y_train)
        test_data = lgb.Dataset(X_test, label=y_test)

        model = lgb.train(
            param, 
            train_data, 
            valid_sets=[test_data], 
            callbacks=[lgb.log_evaluation(period=200)] 
        )
        y_pred_proba = model.predict(X_test, num_iteration=model.best_iteration)

        # AUC計算
        auc = roc_auc_score(y_test, y_pred_proba)

        return auc

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=NT)

    best_lgbm_params = study.best_trial.params

    # 最適化されたパラメータを出力
    print("Best trial:")
    trial = study.best_trial
    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    lgbm_clf = LGBMClassifier(**best_lgbm_params)

    def cb_objective(trial):
        param = {
            'loss_function': 'Logloss',
            'eval_metric': 'AUC',
            'verbose': False,
            'learning_rate': trial.suggest_float('learning_rate', 0.04, 0.08), 
            'iterations': trial.suggest_int('iterations', 600, 750), 
            'depth': trial.suggest_int('depth', 5, 10), 
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 2.0, 3.0), 
            'border_count': trial.suggest_int('border_count', 200, 300), 
            'bagging_temperature': trial.suggest_float('bagging_temperature', 0.7, 0.9) 
        }


        clf = cb.CatBoostClassifier(**param)
        clf.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=50)

        y_pred_proba = clf.predict_proba(X_test)[:,1]
        auc = roc_auc_score(y_test, y_pred_proba)
        return auc

    cb_study = optuna.create_study(direction='maximize')
    cb_study.optimize(cb_objective, n_trials=NT)

    print("Best trial for CatBoost:")
    print(f"  Value: {cb_study.best_trial.value}")
    print("  Params: ")
    for key, value in cb_study.best_trial.params.items():
        print(f"    {key}: {value}")


    best_cat_params = cb_study.best_trial.params
    cat_clf = CatBoostClassifier(**best_cat_params)


    lgbm_clf.fit(X_train, y_train)

    y_pred = lgbm_clf.predict(X_test)
    y_pred_proba_lgbm = lgbm_clf.predict_proba(X_test)[:, 1]

    cat_clf.fit(X_train, y_train)

    y_pred = cat_clf.predict(X_test)
    y_pred_proba_cat = cat_clf.predict_proba(X_test)[:, 1]

    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_lgbm)
    roc_auc_lgb = auc(fpr, tpr)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_cat)
    roc_auc_cat = auc(fpr, tpr)

    ensemble1 = (y_pred_proba_lgbm + y_pred_proba_cat) / 2
    fpr, tpr, thresholds = roc_curve(y_test, ensemble1)
    roc_auc_ensemble1 = auc(fpr, tpr)

    print(roc_auc_lgb,roc_auc_cat,roc_auc_ensemble1)

    df_test = df_temp.loc[df['train_or_valid'] == 'test'].copy()

    X_test = df_test.drop(['train_or_valid','target'], axis=1)
    X_test = X_test.reset_index(drop=True)

    X_test = X_test[X.columns]

    X_test.to_pickle(dir_path + 'X_test.pkl')
    df_test.to_pickle(dir_path + 'df_test.pkl')
    test_index_o.to_pickle(dir_path + 'test_index_o.pkl')

    with open(dir_path + f"lgbm_clf_{r}_{roc_auc_ensemble1*10000:.0f}.pkl", "wb") as file:
        pickle.dump(lgbm_clf, file)

    with open(dir_path + f"cat_clf_{r}_{roc_auc_ensemble1*10000:.0f}.pkl", "wb") as file:
        pickle.dump(cat_clf, file)



[I 2024-08-13 11:21:23,281] A new study created in memory with name: no-name-ffe31aee-f3bf-445d-bea4-25999eb45dfd
[I 2024-08-13 11:21:23,394] Trial 0 finished with value: 0.79497979590439 and parameters: {'lambda_l1': 1.002997343934328, 'lambda_l2': 0.09793600503122728, 'num_leaves': 330, 'feature_fraction': 0.4298567207106613, 'bagging_fraction': 0.592461570033869, 'bagging_freq': 1, 'min_child_samples': 99}. Best is trial 0 with value: 0.79497979590439.


Borderline-SMOTE resampled data shape: (1566, 55) (1566,)


[I 2024-08-13 11:21:23,495] Trial 1 finished with value: 0.7916581056092049 and parameters: {'lambda_l1': 1.0890512302676867, 'lambda_l2': 0.16212419745005474, 'num_leaves': 252, 'feature_fraction': 0.5111479024449622, 'bagging_fraction': 0.585559748018499, 'bagging_freq': 1, 'min_child_samples': 97}. Best is trial 0 with value: 0.79497979590439.
[I 2024-08-13 11:21:23,649] Trial 2 finished with value: 0.7959728785699609 and parameters: {'lambda_l1': 0.8241559389999292, 'lambda_l2': 0.12661037897120503, 'num_leaves': 193, 'feature_fraction': 0.4081036493155741, 'bagging_fraction': 0.634119446400168, 'bagging_freq': 1, 'min_child_samples': 100}. Best is trial 2 with value: 0.7959728785699609.
[I 2024-08-13 11:21:23,785] Trial 3 finished with value: 0.786110540373947 and parameters: {'lambda_l1': 0.8900759817364536, 'lambda_l2': 0.09387669121072564, 'num_leaves': 319, 'feature_fraction': 0.5329586622664638, 'bagging_fraction': 0.6827426431703991, 'bagging_freq': 1, 'min_child_samples': 9

Best trial:
  Value: 0.7959728785699609
  Params: 
    lambda_l1: 0.8241559389999292
    lambda_l2: 0.12661037897120503
    num_leaves: 193
    feature_fraction: 0.4081036493155741
    bagging_fraction: 0.634119446400168
    bagging_freq: 1
    min_child_samples: 100


[I 2024-08-13 11:21:25,033] Trial 0 finished with value: 0.78117937127594 and parameters: {'learning_rate': 0.04777665211636633, 'iterations': 648, 'depth': 8, 'l2_leaf_reg': 2.3939439837806273, 'border_count': 213, 'bagging_temperature': 0.7006593342242662}. Best is trial 0 with value: 0.78117937127594.
[I 2024-08-13 11:21:26,442] Trial 1 finished with value: 0.7910759537018012 and parameters: {'learning_rate': 0.06935253912172631, 'iterations': 690, 'depth': 8, 'l2_leaf_reg': 2.927242984664848, 'border_count': 300, 'bagging_temperature': 0.7708406929324486}. Best is trial 1 with value: 0.7910759537018012.
[I 2024-08-13 11:21:27,439] Trial 2 finished with value: 0.7839189096637218 and parameters: {'learning_rate': 0.06150368207508916, 'iterations': 611, 'depth': 6, 'l2_leaf_reg': 2.5883235322358282, 'border_count': 287, 'bagging_temperature': 0.758352185359831}. Best is trial 1 with value: 0.7910759537018012.
[I 2024-08-13 11:21:30,911] Trial 3 finished with value: 0.7739538387781659 

...

[I 2024-08-13 11:23:05,383] A new study created in memory with name: no-name-1c63ec9f-cd19-4b8d-99f3-eb0950e89422


Borderline-SMOTE resampled data shape: (1558, 55) (1558,)


[I 2024-08-13 11:23:05,769] Trial 0 finished with value: 0.8009287925696594 and parameters: {'lambda_l1': 0.8054268055449216, 'lambda_l2': 0.14726991629543454, 'num_leaves': 314, 'feature_fraction': 0.3511542672356125, 'bagging_fraction': 0.6992871516732082, 'bagging_freq': 2, 'min_child_samples': 85}. Best is trial 0 with value: 0.8009287925696594.
[I 2024-08-13 11:23:05,995] Trial 1 finished with value: 0.8058479532163743 and parameters: {'lambda_l1': 1.0972146590421463, 'lambda_l2': 0.16326702659834674, 'num_leaves': 205, 'feature_fraction': 0.3723720418343749, 'bagging_fraction': 0.6084550989729058, 'bagging_freq': 2, 'min_child_samples': 86}. Best is trial 1 with value: 0.8058479532163743.
[I 2024-08-13 11:23:06,185] Trial 2 finished with value: 0.803921568627451 and parameters: {'lambda_l1': 0.7633665337653112, 'lambda_l2': 0.12244604866445438, 'num_leaves': 215, 'feature_fraction': 0.4528280538898845, 'bagging_fraction': 0.6166119039675858, 'bagging_freq': 2, 'min_child_samples'

Best trial:
  Value: 0.8058479532163743
  Params: 
    lambda_l1: 1.0972146590421463
    lambda_l2: 0.16326702659834674
    num_leaves: 205
    feature_fraction: 0.3723720418343749
    bagging_fraction: 0.6084550989729058
    bagging_freq: 2
    min_child_samples: 86


[I 2024-08-13 11:23:07,994] Trial 0 finished with value: 0.8113519091847264 and parameters: {'learning_rate': 0.045862988636015574, 'iterations': 613, 'depth': 7, 'l2_leaf_reg': 2.168497405681441, 'border_count': 247, 'bagging_temperature': 0.7038583864220689}. Best is trial 0 with value: 0.8113519091847264.
[I 2024-08-13 11:23:09,263] Trial 1 finished with value: 0.81468868249054 and parameters: {'learning_rate': 0.06769498012899731, 'iterations': 714, 'depth': 7, 'l2_leaf_reg': 2.4335522982360533, 'border_count': 280, 'bagging_temperature': 0.7363650824620681}. Best is trial 1 with value: 0.81468868249054.
